In [1]:
from guardrails import Guard
from prompt import PROMPT, categorias
from guardrails.hub import ContainsString
from guard import CSVOutput
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
guard = Guard.for_pydantic(output_class=CSVOutput)

res = guard(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": PROMPT
    }]
)

print(f"{res.validated_output}")


{'data': [{'date': '2024-11-15', 'title': 'Supermercado Central', 'amount': 250.75, 'category': 'supermercado'}, {'date': '2024-11-17', 'title': 'Restaurante Sabor Caseiro', 'amount': 120.9, 'category': 'restaurante'}, {'date': '2024-11-18', 'title': 'Posto Alfa Combustíveis', 'amount': 315.6, 'category': 'transporte'}, {'date': '2024-11-19', 'title': 'Farmácia Bem-Estar', 'amount': 89.45, 'category': 'saude'}, {'date': '2024-11-20', 'title': 'EletroShop', 'amount': 1299.99, 'category': 'eletronicos'}, {'date': '2024-11-21', 'title': 'Livraria Cultura Viva', 'amount': 58.3, 'category': 'educacao'}, {'date': '2024-11-22', 'title': 'Academia Boa Forma', 'amount': 199.0, 'category': 'saude'}, {'date': '2024-11-23', 'title': 'Loja de Roupas Elegance', 'amount': 489.9, 'category': 'vestuario'}, {'date': '2024-11-24', 'title': 'Cinema Top Mall', 'amount': 65.5, 'category': 'lazer'}, {'date': '2024-11-25', 'title': 'Padaria Doce Pão', 'amount': 34.2, 'category': 'supermercado'}, {'date': '202

/home/rafaelcalassara/ufg/01-eprompt/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [5]:
res.validated_output['data'].append({
    'date': '2024-11-15',
    'title': 'Supermercado Central',
    'amount': 250.75,
    'category': 'teste_fail'})

In [3]:

for line in res.validated_output['data']:
    guard = Guard().use(ContainsString, substring=line['category'], on_fail="exception")
    try:
        guard.validate(','.join(categorias))
    except Exception as e:
        print(e)

Validation failed for field with errors: casa,educacao,eletronicos,lazer,restaurante,saude,servicos,supermercado,transporte,vestuario,viagem doesn't contain ignorado
Validation failed for field with errors: casa,educacao,eletronicos,lazer,restaurante,saude,servicos,supermercado,transporte,vestuario,viagem doesn't contain ignorado


In [4]:
res.validated_output['data']

[{'date': '2024-11-15',
  'title': 'Supermercado Central',
  'amount': 250.75,
  'category': 'supermercado'},
 {'date': '2024-11-17',
  'title': 'Restaurante Sabor Caseiro',
  'amount': 120.9,
  'category': 'restaurante'},
 {'date': '2024-11-18',
  'title': 'Posto Alfa Combustíveis',
  'amount': 315.6,
  'category': 'transporte'},
 {'date': '2024-11-19',
  'title': 'Farmácia Bem-Estar',
  'amount': 89.45,
  'category': 'saude'},
 {'date': '2024-11-20',
  'title': 'EletroShop',
  'amount': 1299.99,
  'category': 'eletronicos'},
 {'date': '2024-11-21',
  'title': 'Livraria Cultura Viva',
  'amount': 58.3,
  'category': 'educacao'},
 {'date': '2024-11-22',
  'title': 'Academia Boa Forma',
  'amount': 199.0,
  'category': 'saude'},
 {'date': '2024-11-23',
  'title': 'Loja de Roupas Elegance',
  'amount': 489.9,
  'category': 'vestuario'},
 {'date': '2024-11-24',
  'title': 'Cinema Top Mall',
  'amount': 65.5,
  'category': 'lazer'},
 {'date': '2024-11-25',
  'title': 'Padaria Doce Pão',
  

In [21]:

for line in res.validated_output['data']:
    guard = Guard().use(ContainsString, substring=line['category'], on_fail="exception")
    try:
        guard.validate(','.join(categorias))
    except Exception as e:
        print(e)

/home/rafaelcalassara/ufg/01-eprompt/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [24]:
from guardrails.validators import (
    FailResult,
    PassResult,
    register_validator,
    ValidationResult,
) 
## With Guardrails AI
@register_validator(name="is-haiku", data_type="string")
def is_haiku(value, metadata):
    if not value or len(value.split("\n")) != 3:
        return FailResult(error_message="This is not a haiku")
    return PassResult()

response = Guard().use(is_haiku)(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": "Write a haiku about AI"}],
)
print(response.validated_output)

Silent circuits hum,  
Wisdom born from coded dreams,  
Future's mind awakes.


/home/rafaelcalassara/ufg/01-eprompt/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [27]:
response.model_dump()

{'call_id': '139791568211152',
 'raw_llm_output': "Silent circuits hum,  \nWisdom born from coded dreams,  \nFuture's mind awakes.",
 'validation_summaries': [],
 'validated_output': "Silent circuits hum,  \nWisdom born from coded dreams,  \nFuture's mind awakes.",
 'reask': None,
 'validation_passed': True,
 'error': None}

In [34]:
@register_validator(name="is-valid-category", data_type="string")
def is_valid_category(value, metadata):
    linhas = CSVOutput(value)
    for linha in linhas:
        if not linha['category'] in ','.join(categorias):
            return FailResult(error_message="Essa categoria não é valida")
    return PassResult()

In [36]:
for line in res.validated_output['data']:
    guard = Guard().use(is_valid_category, value=line)
    try:
        guard.validate()
    except Exception as e:
        print(e)

Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() missing 1 required positional argument: 'llm_output'
Guard.validate() mis